This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exchange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [5]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse into the JSON structure that will be returned
# Inspect the JSON structure of the object you created, and take note of how nested it is, as well as the overall structure

import os
import json
import requests
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

params = {'api_key': API_KEY}
responseBody = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json', params=params).text

print(json.loads(responseBody)['dataset_data']['data'][0])

['2020-12-01', 112.2, 112.2, 111.5, 112.0, None, 51.0, 5703.0, None, None, None]


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)



In [8]:
import os
import json
import requests
from pydantic import BaseModel  # For runtime type-checking
from dotenv import load_dotenv
from typing import Union, Optional, List

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')


class DailyTickerSummary(BaseModel):
    opening_price: Optional[float]
    closing_price: Optional[float]
    traded_volume: Optional[float]
    price_change: Optional[float]

    def __str__(self):
        return json.dumps({
            'opening_price': self.opening_price,
            'closing_price': self.closing_price,
            'traded_volume': self.traded_volume,
            'price_change': self.price_change,
        }, indent=2)


def fetch_daily_ticker_summaries(year: Union[int, str]) -> List[DailyTickerSummary]:
    try:
        params = {'api_key': API_KEY, 'start_date': f'{year}-01-01', 'end_date': f'{year}-12-31'}
        response = requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json', params)
        daily_ticker_data: List[List[float]] = json.loads(response.text)['dataset_data']['data']

        return list(map(build_daily_ticker_summary, daily_ticker_data))

    except requests.exceptions.RequestException:
        print('The HTTP request failed')
    except KeyError:
        print('The response body is malformed')

    return []


def build_daily_ticker_summary(price_data: List[float]) -> DailyTickerSummary:
    opening_price = price_data[1] or -1
    closing_price = price_data[4] or -1
    traded_volume = price_data[6] or -1
    low_price = price_data[3]
    high_price = price_data[2]
    price_change = high_price - low_price if None not in [low_price and high_price] else -1

    return DailyTickerSummary(
        opening_price=opening_price,
        closing_price=closing_price,
        traded_volume=traded_volume,
        price_change=price_change
    )


def extract_valid_opening_prices(summaries: List[DailyTickerSummary]) -> List[float]:
    opening_prices = map(lambda summary: summary.opening_price, summaries)
    return list(filter(lambda price: price >= 0, opening_prices))


def extract_valid_daily_trading_volumes(summaries: List[DailyTickerSummary]) -> List[float]:
    volumes = map(lambda summary: summary.traded_volume, summaries)
    return list(filter(lambda volume: volume >= 0, volumes))


def calculate_highest_opening_price(summaries: List[DailyTickerSummary]) -> float:
    return max(extract_valid_opening_prices(summaries))


def calculate_lowest_opening_price(summaries: List[DailyTickerSummary]) -> float:
    return min(extract_valid_opening_prices(summaries))


def calculate_greatest_single_day_delta(summaries: List[DailyTickerSummary]) -> float:
    return max(list(map(lambda summary: summary.price_change, summaries)))


def calculate_average_daily_trading_volume(summaries: List[DailyTickerSummary]) -> float:
    volumes = extract_valid_daily_trading_volumes(summaries)
    return sum(volumes) / len(volumes)


def calculate_median_daily_trading_volume(summaries: List[DailyTickerSummary]) -> float:
    volumes = sorted(extract_valid_daily_trading_volumes(summaries))
    length = len(volumes)
    index = (length - 1) // 2

    if length % 2:
        return volumes[index]
    else:
        return (volumes[index] + volumes[index + 1]) / 2.0


def calculate_greatest_two_day_closing_delta(summaries: List[DailyTickerSummary]) -> float:
    summary_count: int = len(summaries)
    daily_closing_price_deltas: List[float] = []

    for index in range(summary_count):
        today_index = summary_count - 1 - index
        yesterday_index = today_index - 1

        if yesterday_index < 0:
            break

        today_closing = summaries[today_index].closing_price
        yesterday_closing = summaries[yesterday_index].closing_price

        if -1 not in [today_closing, yesterday_closing]:
            closing_price_delta = abs(today_closing - yesterday_closing)
            daily_closing_price_deltas.append(closing_price_delta)

    return max(daily_closing_price_deltas)


def analyze_daily_ticker_data(year: Union[int, str]):
    summaries = fetch_daily_ticker_summaries(year)
    lowest_opening_price = calculate_lowest_opening_price(summaries)
    highest_opening_price = calculate_highest_opening_price(summaries)
    greatest_single_day_price_delta = calculate_greatest_single_day_delta(summaries)
    greatest_two_day_closing_price_delta = calculate_greatest_two_day_closing_delta(summaries)
    average_daily_trading_volume = calculate_average_daily_trading_volume(summaries)
    median_daily_trading_volume = calculate_median_daily_trading_volume(summaries)

    print(f'Lowest Opening Price: {round(lowest_opening_price, 2)}')
    print(f'Highest Opening Price: {round(highest_opening_price, 2)}')
    print(f'Greatest Single Day Price Delta: {round(greatest_single_day_price_delta, 2)}')
    print(f'Greatest 2-Day Closing Price Delta: {round(greatest_two_day_closing_price_delta, 2)}')
    print(f'Average Daily Trading Volume: {round(average_daily_trading_volume, 2)}')
    print(f'Median Daily Trading Volume: {round(median_daily_trading_volume, 2)}')


print("----- AFX_X Stock Ticker Analysis -----\n")
analyze_daily_ticker_data(year=2017)
print("\n---------------------------------------")

----- AFX_X Stock Ticker Analysis -----

{
  "opening_price": 111,
  "closing_price": 2.0,
  "traded_volume": 3.0,
  "price_change": 4.0
}
{
  "opening_price": 111,
  "closing_price": 2.0,
  "traded_volume": 3.0,
  "price_change": 4.0
}

---------------------------------------
